In [6]:
import importlib
import gmail as g
importlib.reload(g) 

gmail = g.GmailConnector()

In [2]:
messages = gmail.list_messages()
print(messages)

msg1 = messages[0]
msg1

[{'id': '18bfd6ad13d964b2', 'threadId': '18bfd6ad13d964b2'}, {'id': '18bfd63c5957988c', 'threadId': '18bfd63c5957988c'}, {'id': '18bfcbc7a052c649', 'threadId': '18bfcbc7a052c649'}, {'id': '18bfca1681e51d79', 'threadId': '18bfca1681e51d79'}, {'id': '18bfc90ce6a70431', 'threadId': '18bfc90ce6a70431'}, {'id': '18bfc830613cb5db', 'threadId': '18bfc830613cb5db'}, {'id': '18bfc5dc3c431980', 'threadId': '18bfc5dc3c431980'}, {'id': '18bfb0de8ae0fb52', 'threadId': '18bfb0de8ae0fb52'}, {'id': '18bf9489bf99d1f0', 'threadId': '18bf9489bf99d1f0'}, {'id': '18bf8f9273a70944', 'threadId': '18bf8f9273a70944'}]


{'id': '18bfd6ad13d964b2', 'threadId': '18bfd6ad13d964b2'}

In [7]:
headers = gmail.headers(msg1['id'])
headers

{'Subject': 'This copywriting GPT is a game changer...',
 'From': 'The ProHuman AI <theprohuman@mail.beehiiv.com>',
 'Date': 'Thu, 23 Nov 2023 18:21:56 +0000 (UTC)'}

In [4]:
from pprint import pprint

text_body = gmail.text_body(msg1['id'])
pprint(text_body)

('**Welcome to the new newsletter…**\n'
 '\n'
 'Most ChatGPT users are creating some lame GPTs…\n'
 '\n'
 'I tried 200+ GPTs on copywriting but they were so lame and generic.\n'
 '\n'
 'I was fed up. So, I created a game-changing copywriting GPT that writes '
 'better than 90% of pros out there.\n'
 '\n'
 'In this post, I’m going to show you how to use it and the link to use it '
 'yourself (it’s free):\n'
 '\n'
 '### Introducing [The '
 'Kopwriter](https://chat.openai.com/g/g-Fp07I6miQ-the-kopywriter):\n'
 '\n'
 'View image: '
 '(https://media.beehiiv.com/cdn-cgi/image/fit=scale-down,format=auto,onerror=redirect,quality=80/uploads/asset/file/dc869a9a-3b12-4066-a454-ae6ab36c8e35/theKOpywriter.png?t=1700762351)\n'
 'Caption: \n'
 '\n'
 'I trained this GPT on 5k+ command text.\n'
 '\n'
 'It can easily generate ad copy for you.\n'
 '\n'
 'What’s the difference between this copywriting assistant and others is that '
 'it asks questions before getting started (which is a unique thing..right

In [8]:
html_body = gmail.html_body(msg1['id'])
pprint(html_body)

('<!DOCTYPE html><html lang="en" xmlns="http://www.w3.org/1999/xhtml" '
 'xmlns:v="urn:schemas-microsoft-com:vml" '
 'xmlns:o="urn:schemas-microsoft-com:office:office" '
 'style="font-size:16px;"><head><meta charset="utf-8"/><!--[if '
 '!mso]><!--><meta http-equiv="X-UA-Compatible" '
 'content="IE=edge"/><!--<![endif]--><meta name="viewport" '
 'content="width=device-width,initial-scale=1"/><meta '
 'name="x-apple-disable-message-reformatting"/><meta name="format-detection" '
 'content="telephone=no,address=no,email=no,date=no,url=no"/><meta '
 'name="color-scheme" content="light"/><meta name="supported-color-schemes" '
 'content="light"/><title>This copywriting GPT is a game '
 'changer...</title><!--[if '
 'mso]><xml><o:OfficeDocumentSettings><o:AllowPNG/><o:PixelsPerInch>96</o:PixelsPerInch></o:OfficeDocumentSettings></xml><![endif]--><style>\n'
 '  :root { color-scheme: light; supported-color-schemes: light; }\n'
 '  body { margin: 0; padding: 0; min-width: 100%!important; '
 '-ms-